In [1]:
import pandas as pd

from dna_classification.models import DNASequenceClassifier
from dna_classification.tokenization import DNATokenizer

from tqdm import tqdm

/Users/nathanleroy/uva/fa2023/nlp/final/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data = pd.read_csv("data/virus.txt", skiprows=1, header=None, names=["sequence", "label"], sep="\t")

In [4]:
tokenizer = DNATokenizer()
tokenizer.build_vocab("data/virus.txt")

model = DNASequenceClassifier(
    vocab_size=tokenizer.vocab_size,
    embedding_dim=192,
    hidden_dim=64,
    num_layers=2,
    num_classes=data["label"].nunique()
)

model.add_tokenizer(tokenizer)

In [ ]:
train_loss, val_loss = model.train_model(
    data=data,
    epochs=100,
    batch_size=256,
    device="cuda",
    optimizer_params={
        "lr": 0.001,
    }
)

In [ ]:
# get accuracy
model.cpu()
model.eval()
correct = 0
total = 0

# subset
data_sample = data.sample(1000)

for i in tqdm(range(len(data_sample))):
    sequence = data.iloc[i]["sequence"]
    label = data.iloc[i]["label"]
    pred = model.predict(sequence)
    if pred == label:
        correct += 1
    total += 1

print(f"Accuracy: {correct / total}")